In [16]:
from pybit.unified_trading import HTTP
from dotenv import load_dotenv
import pandas as pd
import os
import pprint
load_dotenv()



session = HTTP(
    testnet=False,
    demo=True,
    api_key=os.getenv('API_DEMO_KEY'),
    api_secret=os.getenv('API_DEMO_SECRET'),
)


def get_account_balance(ticker):
    # Retrieve account balance from Bybit
    response = session.get_wallet_balance(accountType="UNIFIED",coin=ticker)  

    
    if response['retCode'] == 0:
        balance = response['result']['list'][0]['coin'][0]['walletBalance']
        print(f'The balance of {ticker} is: {balance}')
        return balance
    else:
        print("Failed to retrieve account balance:", response['ret_msg'])
        return None


def fetch_market_data(symbol):
    data = session.get_kline(symbol=symbol, interval=1)

    df = pd.DataFrame(data['result']['list'])
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']
    df['Date'] = pd.to_datetime(pd.to_numeric(df['Date']), unit='ms')

    df.set_index('Date', inplace=True)

    df = df[::-1]
    return df


def place_order(symbol, side, qty_percentage, price, stop_loss=None, take_profit=None, leverage=10):
    balance = float(get_account_balance('USDT'))
    qty_before_lev = (balance*qty_percentage)/float(price)
    qty_final = round(qty_before_lev*leverage,3)
    print(session.place_order(
        category = "linear",
        symbol = symbol,
        side = side,
        orderType = "Limit",
        qty = str(qty_final),
        price=price,
        take_profit = take_profit,
        stop_loss = stop_loss,
        tpslMode = 'Full'

    ))


def calculate_indicators(df, window=14):
    #METHOD 1 RSI
    change = df["Close"].astype(float).diff()
    change.dropna(inplace = True)
    change_up = change.copy()
    change_down = change.copy()
    change_up[change_up<0]=0
    change_down[change_down>0]=0
    change.equals(change_up+change_down)
    avg_up = change_up.rolling(14).mean()
    avg_down = change_down.rolling(14).mean().abs()
    df['rsi'] = 100* avg_up/(avg_up+avg_down)


    #METHOD 2 RSI
    delta = df['Close'].astype(float).diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=12).mean()
    avg_loss = loss.rolling(window=12).mean()
    rs = avg_gain / avg_loss
    df['rsi2'] = 100 - (100 / (1 + rs))
    df['rsi_sma'] = df['rsi'].rolling(window=14).mean()



    # MACD
    df['EMA12']= df['Close'].astype(float).ewm(span=12).mean() #SHORT EMA
    df['EMA26']= df['Close'].astype(float).ewm(span=26).mean()  #LONG EMA
    df['macd']= df['EMA12']-df['EMA26']
    df['macd_signal'] = df['macd'].ewm(span=9).mean()

    return df


    
symbol = 'BTCUSDT'
data = fetch_market_data(symbol)
new = calculate_indicators(data)
display(new)
'''
print(session.set_leverage(
    category="linear",
    symbol="BTCUSDT",
    buyLeverage="10",
    sellLeverage="10",
))
'''

get_account_balance('USDT')
#place_order('BTCUSDT', "Sell", 0.05, "110000", "120000", "100000", 10)
pprint.pp(session.get_open_orders(
    category="linear",
    symbol="BTCUSDT",
    openOnly=0,
    limit=1,
))


,Open,High,Low,Close,Volume,Turnover,rsi,rsi2,rsi_sma,EMA12,EMA26,macd,macd_signal
Date,,,,,,,,,,,,,
2025-01-20 08:25:00,108250.3,108250.3,108148.9,108164.7,53.028,5736812.0112,NaN,NaN,NaN,108164.700000,108164.700000,0.000000,0.000000
2025-01-20 08:26:00,108164.7,108247.5,108084.3,108093.9,63.609,6879014.1944,NaN,NaN,NaN,108126.350000,108127.938462,-1.588462,-0.882479
2025-01-20 08:27:00,108093.9,108161.9,108064.5,108072.7,86.968,9401998.7217,NaN,NaN,NaN,108105.410393,108108.091819,-2.681426,-1.619752
2025-01-20 08:28:00,108072.7,108089.1,107977.2,108087.7,216.681,23405633.3526,NaN,NaN,NaN,108099.819914,108102.391157,-2.571243,-1.942073
2025-01-20 08:29:00,108087.7,108131.2,108051.5,108052.6,48.165,5206187.7423,NaN,NaN,NaN,108086.990432,108090.844383,-3.853951,-2.510813
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-20 11:40:00,108219.8,108241.2,108180,108227.9,26.79,2899217.2499,34.580052,30.356787,53.465350,108332.602918,108319.207756,13.395162,43.708607
2025-01-20 11:41:00,108227.9,108243.8,108154.4,108183.6,37.035,4006714.8818,34.871795,24.273899,51.512624,108309.679392,108309.162734,0.516658,35.070217
2025-01-20 11:42:00,108183.6,108300,108175.7,108299.9,28.258,3058295.446,40.555639,42.027730,50.029137,108308.174870,108308.476606,-0.301735,27.995827


The balance of USDT is: 100070
{'retCode': 0,
 'retMsg': 'OK',
 'result': {'nextPageCursor': '1db872f6-9d72-4723-8b40-62eaa662f61d%3A1737372449550%2C1db872f6-9d72-4723-8b40-62eaa662f61d%3A1737372449550',
            'category': 'linear',
            'list': [{'symbol': 'BTCUSDT',
                      'orderType': 'Limit',
                      'orderLinkId': '',
                      'slLimitPrice': '0',
                      'orderId': '1db872f6-9d72-4723-8b40-62eaa662f61d',
                      'cancelType': 'UNKNOWN',
                      'avgPrice': '',
                      'stopOrderType': '',
                      'lastPriceOnCreated': '108326.2',
                      'orderStatus': 'New',
                      'createType': 'CreateByUser',
                      'takeProfit': '100000',
                      'cumExecValue': '0',
                      'tpslMode': 'Full',
                      'smpType': 'None',
                      'triggerDirection': 0,
                     

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf

def add_trading_signals(df):
    df['signal'] = 0

    for i in range(1, len(df)):
        if df['rsi'].iloc[i] < 30 and df['rsi'].iloc[i - 1] >= 30:
            df.loc[df.index[i], 'signal'] = 1  # Buy signal
        elif df['rsi'].iloc[i] > 70 and df['rsi'].iloc[i - 1] <= 70:
            df.loc[df.index[i], 'signal'] = -1  # Sell signal
    
    return df

# Apply trading signals to the data
data_with_signals = add_trading_signals(new)

# Display updated data
display(data_with_signals)

# Plot buy/sell signals on the RSI chart
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(data_with_signals.index, data_with_signals['rsi'], label='RSI', color='orange')
ax.axhline(30, linestyle='--', color='green', linewidth=1.5)
ax.axhline(70, linestyle='--', color='red', linewidth=1.5)

# Mark buy signals
buy_signals = data_with_signals[data_with_signals['signal'] == 1]
sell_signals = data_with_signals[data_with_signals['signal'] == -1]
ax.scatter(buy_signals.index, buy_signals['rsi'], marker='^', color='green', label='Buy Signal', alpha=1)
ax.scatter(sell_signals.index, sell_signals['rsi'], marker='v', color='red', label='Sell Signal', alpha=1)

ax.set_title('RSI with Buy and Sell Signals')
ax.legend()
plt.tight_layout()
plt.show()
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (16, 16)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=True, figsize=(16, 16))  



candlestick_data = data[['Open', 'High', 'Low', 'Close']].astype(float) 
mpf.plot(
    candlestick_data,
    type='candle',
    ax=ax1,
    style='charles',
    mav=(12,),  # Moving average, using 12-Day EMA
    volume=False, 
    show_nontrading=True
)
ax1.set_title(f'{symbol} Candlestick Chart with 12-Day EMA and Volume')


# Second chart: Plot the RSI
ax2.set_title('Relative Strength Index')
ax2.plot(data['rsi'], color='orange', linewidth=1)
ax2.axhline(30, linestyle='--', linewidth=1.5, color='green')  # Oversold level
ax2.axhline(70, linestyle='--', linewidth=1.5, color='red')    # Overbought level

#3
ax3.set_title('Relative Strength Index 2')
ax3.plot(data['rsi2'], color='orange', linewidth=1)
ax3.axhline(30, linestyle='--', linewidth=1.5, color='green')  # Oversold level
ax3.axhline(70, linestyle='--', linewidth=1.5, color='red')    # Overbought level


ax4.set_title('MACD and MACD Signal')
ax4.plot(data['macd'], label='MACD', color='blue', linewidth=1)
ax4.plot(data['macd_signal'], label='Signal Line', color='red', linewidth=1)
ax4.axhline(0, linestyle='--', color='black', linewidth=1)  
ax4.legend(loc='upper left')


ax3.set_ylim(0, 90)  



plt.tight_layout()  
plt.show()
